# **Generation Experiment**

In [ ]:
%pip install --quiet --upgrade bitsandbytes langchain langchain-community langchain-huggingface transformers beautifulsoup4 faiss-gpu rank_bm25 lark qdrant-client langchain-chroma langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 109.3 MB/s eta 0:00:

In [ ]:
from langchain_core.documents import Document
from langchain.chains.query_constructor.base import AttributeInfo
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig
from langchain_core.output_parsers import StrOutputParser
import re
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from operator import itemgetter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.retrievers import EnsembleRetriever # Supports Ensembling of results from multiple retrievers
from langchain_community.retrievers import BM25Retriever
from pydantic import BaseModel, Field
from typing import List
from langchain_text_splitters import RecursiveCharacterTextSplitter
from google.colab import userdata
from langchain import PromptTemplate
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
import os
import json
from google.colab import files
import time
from langchain_groq import ChatGroq

### **Query Decomposition:**

Decomposition is a query re-writing technique that focuses on decomposing a question into a set of subquestions.

This is applicable and effective for our use case as users planning a holiday tend to string together many requests in a single query. By breaking down a large queries into sub-queries, the retriever can retrieve more relevant documents to each sub-query and therefore, support the LLM in answering the whole query better

In [ ]:
# Prompt Decomposition template used by the LLM to help break a question into sub questions

template = """You are an expert at converting user travel questions into sub-questions. \
You have access to several documents about the different travel destinations. \

Perform query decomposition. Given a user question, break it down into the most specific sub questions you can \
which will help you answer the original question. Each sub question should be about a single concept/fact/idea.

If there are acronyms or words you are not familiar with, do not try to rephrase them.
"""

template_2 = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries each on a new line):"""

prompt_decomposition = PromptTemplate.from_template(template)

In [ ]:
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
llm = ChatGroq()

**Test Queries for Decomposition**

In [ ]:
test_query_1 = "When is the best time to go Finland and what is there to do"

<u>Decompsition of a question into sub-questions</u>

In [ ]:
# Apply the decompsition template and break down the questions into sub questions using the prompt decompsition pipeline
questions = generate_queries_decomposition.invoke({"question":test_query_1})

In [ ]:
questions

['1) when is the best time to visit finland for tourism?',
 '2) what activities are available in finland during different seasons?',
 '3) how does weather affect tourist attractions in finland?']

### **Fix Best Retriever from Retreiver Evaluation**

In [ ]:
# Initialise the FAISS retriever
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world")))
faiss_vector_store = FAISS(
    embedding_function=embeddings_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
faiss_vector_store.add_documents(docs)
retriever = faiss_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # num docs to return from FAISS

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<u>Build the Final Answer Recursively</u>

In [ ]:
'''
Prompt template to recursively answer sub questions and build up the answers
Might need to modify prompt to ask it to use only the context

question: sub-question to be answered
q_a_pairs: Built up question-answer pairs that might be relevant
context: context retrieved for the current sub-question
Idea is to recursively answer each sub-question, using the current context and building upon previous answers to provide more comprehensive responses.
'''
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question:

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""
decomposition_prompt = PromptTemplate.from_template(template)

In [ ]:
# Utility function to format a given question and answer
def format_qa_pair(question, answer):
    """Format Q and A pair"""
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

# Initialise the q_a_pairs to be empty at first
q_a_pairs = ""

# For each sub-question that we decomposed from our main question earlier
for q in questions:
  rag_chain = (
  # Given {"question":q,"q_a_pairs":q_a_pairs}
  {"context": itemgetter("question") | retriever,  # Get the context relevant to the subquestion using the retriever
    "question": itemgetter("question"), # Get the subquestion
    "q_a_pairs": itemgetter("q_a_pairs")} # Get the built up qna pairs
  | decomposition_prompt # Pass the arguments into the template
  | llm.bind(skip_prompt=True)
  | StrOutputParser()) # Get the result from the LLM

  # Pass our rag chain the sub question and any prev built up q_a_pairs
  answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
  q_a_pair = format_qa_pair(q,answer) # Format it as sub_question, answer
  q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair # Update/Build the q_a_pairs

In [ ]:
answer

"To address the question of how weather affects tourist attractions in Finland, we will consider the provided context and background information.\n\nFirstly, let's examine the details about accommodations in Finland:\n\n1. **Snow Activities**: The Glass huts in Skyfire village in Rovaniemi, Lapland are highlighted as one of the best places to stay. These glass huts provide a unique experience where tourists can observe the Northern Lights and enjoy the snowy landscape. This indicates that the presence of snow and the occurrence of the Northern Lights significantly influence the appeal and accessibility of this attraction.\n\n2. **Northern Lights Viewing**: The Northern Lights, also known as Aurora Borealis, are mentioned as a captivating phenomenon that draws tourists to Finland. Snowfall conditions are crucial for visibility because they create a clear sky conducive to observing the aurora. Without sufficient snow cover, the atmosphere might be too cloudy or polluted, reducing the cha

<u>Build the Final Answer Individually</u>

In [ ]:
# Answer each sub-question individually
# RAG prompt
'''
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question}

Context: {context}

Answer:
'''
prompt_rag = hub.pull("rlm/rag-prompt")
def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """Perform RAG on each sub-question"""
    # Generate the sub questions using the chain
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    # Initialize a list to hold RAG results of each sub-question
    rag_results = []
    for sub_question in sub_questions:
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)
        # Use retrieved documents and sub-question to answer the sub question
        answer = (prompt_rag | llm.bind(skip_prompt=True) | StrOutputParser()).invoke({"context": retrieved_docs,
                                                                "question": sub_question})
        # Append the answer to the sub question
        rag_results.append(answer)
    # Return the list of sub questions and their answers
    return rag_results,sub_questions

answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
<ipython-input-38-9197ef3514dd>:32: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(sub_question)


In [ ]:
# Utility function to format a given question and answer
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

# Format the list of sub questions and their answers from just now
context = format_qa_pairs(questions, answers)

# Prompt template to use each individual sub-question and answer, as well as the main question
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = PromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm.bind(skip_prompt=True)
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

"To answer your question, the best time to visit Finland for tourism is during winter months like December through February when the Northern Lights (auroras) are most visible due to long nights and cold temperatures. This period provides opportunities for activities like snowshoeing, skiing, and aurora watching. However, summer from June to August also offers unique experiences such as midnight sun and vibrant nature with activities including hiking, kayaking, and berry-picking. Additionally, there's a chance to experience the country's unique glass huts in Lapland during the winter season. Overall, both winter and summer offer distinct and memorable experiences in Finland. To get the most out of your trip, it's recommended to check local forecasts and plan accordingly based on your interests and preferences. Here’s a summary of the key points:\n\n- **Best Time to Visit:** Winter (December through February) for Northern Lights and snowy landscapes.\n- **Summer Activities:** Hiking, ka

**Conclusion**: To be filled in